In [26]:
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
import numpy as np
from pymongo import MongoClient
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [27]:
#url = 'E:/temp_ml_data/teste_binelo.csv'
#data = pd.read_csv(url, delimiter=',')
#entradas=data.iloc[:,0:2]
#saidas = data.iloc[:,2:5]
#saidas

In [54]:
client = MongoClient('mongodb://localhost:27017')
db = client.admin
status = db.command("serverStatus")
mydb = client["tcc"]
mycol = mydb["online_retail"]
myquery = { "$or": [{ "modeProd": "1"  }, { "modeProd": "119616" }, 
                     { "modeProd": "84879" }, { "modeProd": "84880" }] }
mydoc = mycol.find(myquery).sort("modeProd")
data = pd.DataFrame(list(mydoc))
data = data[:1600]
data[:1600]

,_id,clientId,modeProd,avgQty,month
0,5f1cca2ec9272432303f99cb,15311,119616,1,1
1,5f1cca2fc9272432303fa49e,1,119616,1,1
2,5f1cca2fc9272432303fc30c,1,119616,1,1
3,5f1cca30c927243230405d68,12539,119616,4,1
4,5f1cca30c92724323040b6e7,12539,119616,4,1
...,...,...,...,...,...
1592,5f1cca3ac92724323046ab1b,15411,84880,1,11
1593,5f1cca3ac92724323046c26d,13614,84880,2,11
1594,5f1cca3bc927243230475d4c,13883,84880,4,11
1595,5f1cca3bc92724323047b7ab,1,84880,1,12


In [55]:
data = data[['clientId', 'month', 'modeProd']]
entradas=data.iloc[:,0:2]
saidas = data.iloc[:,2:3]
entradas = entradas.astype("int32")
saidas = saidas.astype("int32")
#print(entradas[:3])
#print(saidas[:3])

In [56]:
onehot_saidas = pd.get_dummies(saidas.modeProd, prefix='prod')
onehot_saidas[:3]

,prod_84879,prod_84880,prod_119616
0,0,0,1
1,0,0,1
2,0,0,1


In [57]:
onehot_entradas = pd.get_dummies(entradas.clientId, prefix='client')
num_inputs = len(onehot_entradas.columns) + 1
print(num_inputs)
onehot_entradas['month'] = entradas.month
onehot_entradas = onehot_entradas.astype('uint8')
onehot_entradas[:3]

722


,client_1,client_12370,client_12381,client_12423,client_12429,client_12444,client_12456,client_12461,client_12473,client_12476,...,client_18222,client_18223,client_18228,client_18239,client_18242,client_18252,client_18260,client_18265,client_18277,month
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [58]:
onehot_entradas.dtypes

client_1        uint8
client_12370    uint8
client_12381    uint8
client_12423    uint8
client_12429    uint8
                ...  
client_18252    uint8
client_18260    uint8
client_18265    uint8
client_18277    uint8
month           uint8
Length: 722, dtype: object

In [59]:
x = saidas['modeProd'].to_numpy()
saidas_units = len(np.unique(x))
saidas_units

3

In [60]:
model = Sequential()

# original: 64 units, relu, 2 inputs
model.add(Dense(units=64, activation='relu', input_dim=num_inputs))
model.add(Dense(units=saidas_units, activation='softmax'))

#sparse_categorical_crossentropy
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(onehot_entradas, onehot_saidas,
          batch_size=10,
          epochs=1000,
          verbose=1,
          validation_data=(onehot_entradas,onehot_saidas))

Epoch 1/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.9361 - val_loss: 0.2912 - val_accuracy: 0.9405
Epoch 2/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2849 - accuracy: 0.9405 - val_loss: 0.2799 - val_accuracy: 0.9405
Epoch 3/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2734 - accuracy: 0.9405 - val_loss: 0.2653 - val_accuracy: 0.9405
Epoch 4/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2627 - accuracy: 0.9405 - val_loss: 0.2579 - val_accuracy: 0.9405
Epoch 5/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2573 - accuracy: 0.9405 - val_loss: 0.2522 - val_accuracy: 0.9405
Epoch 6/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2533 - accuracy: 0.9405 - val_loss: 0.2481 - val_accuracy: 0.9405
Epoch 7/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2469 - accuracy: 0.9405 - val_loss: 0.2447 - val_accuracy:

160/160 [==============================] - 0s 1ms/step - loss: 0.2150 - accuracy: 0.9405 - val_loss: 0.2136 - val_accuracy: 0.9405
Epoch 58/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2155 - accuracy: 0.9405 - val_loss: 0.2098 - val_accuracy: 0.9405
Epoch 59/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.2165 - accuracy: 0.9405 - val_loss: 0.2095 - val_accuracy: 0.9405
Epoch 60/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.2121 - accuracy: 0.9405 - val_loss: 0.2135 - val_accuracy: 0.9405
Epoch 61/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.9405 - val_loss: 0.2092 - val_accuracy: 0.9405
Epoch 62/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.2128 - accuracy: 0.9405 - val_loss: 0.2080 - val_accuracy: 0.9405
Epoch 63/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.2137 - accuracy: 0.9405 - val_loss: 0.2134 - val_accuracy: 0.9405

160/160 [==============================] - 0s 1ms/step - loss: 0.1843 - accuracy: 0.9418 - val_loss: 0.1965 - val_accuracy: 0.9405
Epoch 114/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.1858 - accuracy: 0.9411 - val_loss: 0.1754 - val_accuracy: 0.9405
Epoch 115/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.1833 - accuracy: 0.9399 - val_loss: 0.2130 - val_accuracy: 0.9430
Epoch 116/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.1833 - accuracy: 0.9405 - val_loss: 0.1902 - val_accuracy: 0.9405
Epoch 117/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.1826 - accuracy: 0.9405 - val_loss: 0.1765 - val_accuracy: 0.9405
Epoch 118/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1821 - accuracy: 0.9411 - val_loss: 0.1729 - val_accuracy: 0.9405
Epoch 119/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1801 - accuracy: 0.9411 - val_loss: 0.2213 - val_accuracy: 

160/160 [==============================] - 0s 2ms/step - loss: 0.1517 - accuracy: 0.9449 - val_loss: 0.1448 - val_accuracy: 0.9436
Epoch 170/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1464 - accuracy: 0.9436 - val_loss: 0.1295 - val_accuracy: 0.9468
Epoch 171/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1413 - accuracy: 0.9487 - val_loss: 0.1469 - val_accuracy: 0.9436
Epoch 172/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1498 - accuracy: 0.9436 - val_loss: 0.1579 - val_accuracy: 0.9436
Epoch 173/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.1505 - accuracy: 0.9436 - val_loss: 0.1259 - val_accuracy: 0.9474
Epoch 174/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1514 - accuracy: 0.9430 - val_loss: 0.1407 - val_accuracy: 0.9436
Epoch 175/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1494 - accuracy: 0.9480 - val_loss: 0.1530 - val_accuracy: 

160/160 [==============================] - 0s 2ms/step - loss: 0.1170 - accuracy: 0.9599 - val_loss: 0.1290 - val_accuracy: 0.9537
Epoch 226/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1298 - accuracy: 0.9537 - val_loss: 0.1252 - val_accuracy: 0.9537
Epoch 227/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1196 - accuracy: 0.9562 - val_loss: 0.1644 - val_accuracy: 0.9524
Epoch 228/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1270 - accuracy: 0.9543 - val_loss: 0.1015 - val_accuracy: 0.9587
Epoch 229/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1246 - accuracy: 0.9543 - val_loss: 0.3677 - val_accuracy: 0.9430
Epoch 230/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1306 - accuracy: 0.9562 - val_loss: 0.1181 - val_accuracy: 0.9537
Epoch 231/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.1216 - accuracy: 0.9524 - val_loss: 0.0929 - val_accuracy: 

...

160/160 [==============================] - 0s 2ms/step - loss: 0.0714 - accuracy: 0.9724 - val_loss: 0.0594 - val_accuracy: 0.9793
Epoch 730/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0687 - accuracy: 0.9743 - val_loss: 0.0524 - val_accuracy: 0.9793
Epoch 731/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0646 - accuracy: 0.9781 - val_loss: 0.0633 - val_accuracy: 0.9781
Epoch 732/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0646 - accuracy: 0.9762 - val_loss: 0.0517 - val_accuracy: 0.9793
Epoch 733/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0686 - accuracy: 0.9737 - val_loss: 0.1687 - val_accuracy: 0.9361
Epoch 734/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0608 - accuracy: 0.9762 - val_loss: 0.0611 - val_accuracy: 0.9756
Epoch 735/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0771 - accuracy: 0.9737 - val_loss: 0.0622 - val_accuracy: 

160/160 [==============================] - 0s 2ms/step - loss: 0.0676 - accuracy: 0.9737 - val_loss: 0.1765 - val_accuracy: 0.9361
Epoch 786/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0673 - accuracy: 0.9750 - val_loss: 0.0510 - val_accuracy: 0.9800
Epoch 787/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0701 - accuracy: 0.9706 - val_loss: 0.0620 - val_accuracy: 0.9775
Epoch 788/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0687 - accuracy: 0.9750 - val_loss: 0.0771 - val_accuracy: 0.9775
Epoch 789/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0629 - accuracy: 0.9743 - val_loss: 0.0613 - val_accuracy: 0.9781
Epoch 790/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0702 - accuracy: 0.9706 - val_loss: 0.0547 - val_accuracy: 0.9793
Epoch 791/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0652 - accuracy: 0.9756 - val_loss: 0.1028 - val_accuracy: 

160/160 [==============================] - 0s 2ms/step - loss: 0.0701 - accuracy: 0.9731 - val_loss: 0.0553 - val_accuracy: 0.9793
Epoch 842/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0685 - accuracy: 0.9743 - val_loss: 0.0657 - val_accuracy: 0.9706
Epoch 843/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0656 - accuracy: 0.9762 - val_loss: 0.0592 - val_accuracy: 0.9781
Epoch 844/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0627 - accuracy: 0.9762 - val_loss: 0.0535 - val_accuracy: 0.9800
Epoch 845/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0610 - accuracy: 0.9768 - val_loss: 0.1949 - val_accuracy: 0.9368
Epoch 846/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0691 - accuracy: 0.9743 - val_loss: 0.0510 - val_accuracy: 0.9800
Epoch 847/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0621 - accuracy: 0.9743 - val_loss: 0.0573 - val_accuracy: 

160/160 [==============================] - 0s 1ms/step - loss: 0.0660 - accuracy: 0.9712 - val_loss: 0.0498 - val_accuracy: 0.9806
Epoch 898/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0576 - accuracy: 0.9762 - val_loss: 0.0527 - val_accuracy: 0.9800
Epoch 899/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0616 - accuracy: 0.9762 - val_loss: 0.0842 - val_accuracy: 0.9587
Epoch 900/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0613 - accuracy: 0.9762 - val_loss: 0.1000 - val_accuracy: 0.9524
Epoch 901/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0524 - accuracy: 0.9806 - val_loss: 0.0496 - val_accuracy: 0.9812
Epoch 902/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0630 - accuracy: 0.9756 - val_loss: 0.0551 - val_accuracy: 0.9800
Epoch 903/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0622 - accuracy: 0.9781 - val_loss: 0.0509 - val_accuracy: 

160/160 [==============================] - 0s 1ms/step - loss: 0.0594 - accuracy: 0.9775 - val_loss: 0.0560 - val_accuracy: 0.9793
Epoch 954/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0619 - accuracy: 0.9768 - val_loss: 0.0619 - val_accuracy: 0.9706
Epoch 955/1000
160/160 [==============================] - ETA: 0s - loss: 0.0650 - accuracy: 0.97 - 0s 2ms/step - loss: 0.0629 - accuracy: 0.9768 - val_loss: 0.0501 - val_accuracy: 0.9800
Epoch 956/1000
160/160 [==============================] - 0s 2ms/step - loss: 0.0650 - accuracy: 0.9756 - val_loss: 0.0522 - val_accuracy: 0.9806
Epoch 957/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0657 - accuracy: 0.9743 - val_loss: 0.0517 - val_accuracy: 0.9793
Epoch 958/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0639 - accuracy: 0.9762 - val_loss: 0.0599 - val_accuracy: 0.9787
Epoch 959/1000
160/160 [==============================] - 0s 1ms/step - loss: 0.0624 - accuracy: 

In [61]:
y=model.predict(onehot_entradas)
predicted = np.round(y)
print(y)
print(predicted)

not_zero = 0
for row in predicted:
   for value in row:
       if value != 0:
            #print(row)
            not_zero += 1
            break
   
if not_zero == 0:
    print("Não convergiu.")
else:
    print("Convergiu: " + str(not_zero))


[[2.6003217e-02 6.3952571e-03 9.6760154e-01]
 [5.9886867e-01 3.6009509e-02 3.6512184e-01]
 [5.9886867e-01 3.6009509e-02 3.6512184e-01]
 ...
 [6.3946366e-02 9.3605369e-01 1.4561528e-13]
 [9.4867420e-01 5.1325746e-02 1.8034712e-14]
 [9.2225894e-02 9.0777409e-01 1.4568696e-14]]
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
Convergiu: 1597


In [70]:
real_outputs = onehot_saidas.to_numpy()
predicted_outputs = predicted.astype("uint32")

In [71]:
with open("E:/temp_ml_data/c7_saidas.txt", "w") as txt_file:
    for line in real_outputs:
        txt_file.write(" ".join(str(line)) + "\n") 

In [72]:
with open("E:/temp_ml_data/c7_predicted.txt", "w") as txt_file:
    for line in predicted_outputs:
        txt_file.write(" ".join(str(line)) + "\n") 

In [78]:
otp_len = len(real_outputs)
otps = range(otp_len)

correct_pred = 0
wrong_pred = 0

for i in otps:
  if(str(real_outputs[i]) == str(predicted_outputs[i])):
    correct_pred += 1
  else:
    wrong_pred += 1

perc = correct_pred / otp_len * 100
print("Correct predictions: " + str(correct_pred))
print("Percentage correct: " +  str(perc))

Correct predictions: 1563
Percentage correct: 97.871008140263
